## Spark DataFrame

In [1]:
from pyspark.sql.session import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

25/04/28 15:31:45 WARN Utils: Your hostname, nguyenphucvinh resolves to a loopback address: 127.0.1.1; using 192.168.1.108 instead (on interface wlo1)
25/04/28 15:31:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/28 15:31:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.sql.types import StructType, StructField, LongType, StringType

# Read `people.json`
lables = [("age", LongType()), ("name", StringType())]
schema = StructType([StructField(x[0], x[1]) for x in lables])
people_json = spark.read.json("./data_lab7/people.json", schema=schema)
people_json.show()

+----+-------+
| age|   name|
+----+-------+
|NULL|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [ ]:
# Show head

Row(age=None, name='Michael')

In [ ]:
people_json.head()

Row(age=None, name='Michael')

In [ ]:
# Show schema

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [73]:
people_json.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [ ]:
# Show column

['age', 'name']

In [74]:
people_json.columns

['age', 'name']

In [ ]:
# Describe

DataFrame[summary: string, age: string, name: string]

In [16]:
data_frame_types = [
    f"{name}: {col_type}" for name, col_type in people_json.describe().dtypes
]
print(f'DataFrame[{", ".join(data_frame_types)}]')

DataFrame[summary: string, age: string, name: string]


In [ ]:
# Describe and show

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



In [92]:
print(people_json.describe())

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   NULL|
| stddev|7.7781745930520225|   NULL|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



In [17]:
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

In [18]:
# age: name of variable, type of variable, True/False: if it can contain null
data_schema = [
    StructField("age", IntegerType(), True),
    StructField("name", StringType(), True),
]

In [19]:
data_schema

[StructField('age', IntegerType(), True),
 StructField('name', StringType(), True)]

In [20]:
final_struc = StructType(fields=data_schema)

In [21]:
final_struc

StructType([StructField('age', IntegerType(), True), StructField('name', StringType(), True)])

In [23]:
df = spark.read.json('./data_lab7/people.json', schema=final_struc)

In [ ]:
df.printSchema()
#Now it has corrected the age as integer instead of long

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



In [ ]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [ ]:
df['age']

Column<b'age'>

In [ ]:
type(df['age'])

pyspark.sql.column.Column

In [ ]:
#If you want to select a dataframe column
df.select('age')

DataFrame[age: int]

In [ ]:
#If you want to view the results of dataframe column
df.select('age').show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



In [ ]:
#If you want to view selected rows of dataframe
df.head(2)

[Row(age=None, name='Michael'), Row(age=30, name='Andy')]

In [ ]:
df.head(2)[0]

Row(age=None, name='Michael')

In [ ]:
df.head(2)[0]['name']

'Michael'

In [ ]:
type(df.head(2)[0])

pyspark.sql.types.Row

In [ ]:
type(df.head(2)[0]['name'])

str

In [ ]:
##The reason there are so many datatypes and objects in Spark because of its ability
#to read from a distributed source and map that out to compute

In [ ]:
#selecting multiple columns


DataFrame[age: int, name: string]

In [ ]:
# Show the dataframe

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [25]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|NULL|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [ ]:
#Creating new columns `newAge`

+----+-------+------+
| age|   name|newAge|
+----+-------+------+
|null|Michael|  null|
|  30|   Andy|    30|
|  19| Justin|    19|
+----+-------+------+



In [28]:
from pyspark.sql.functions import col

new_df = df.withColumn("newAge", col=col('age'))
print(new_df)

+----+-------+------+
| age|   name|newAge|
+----+-------+------+
|NULL|Michael|  NULL|
|  30|   Andy|    30|
|  19| Justin|    19|
+----+-------+------+



In [ ]:
#Creating new columns `doubleAge`

+----+-------+---------+
| age|   name|doubleAge|
+----+-------+---------+
|null|Michael|     null|
|  30|   Andy|       60|
|  19| Justin|       38|
+----+-------+---------+



In [29]:
from pyspark.sql.functions import lit

new_df = df.withColumn("doubleAge", col("age") * lit(2))
print(new_df)

+----+-------+---------+
| age|   name|doubleAge|
+----+-------+---------+
|NULL|Michael|     NULL|
|  30|   Andy|       60|
|  19| Justin|       38|
+----+-------+---------+



In [ ]:
#The above operations are not inplace so everytime you make
#any changes you have to save them to a new variable

In [ ]:
#Renaming a column 'age' by 'my_new_age'

+----------+-------+
|my_new_age|   name|
+----------+-------+
|      null|Michael|
|        30|   Andy|
|        19| Justin|
+----------+-------+



In [30]:
new_df = df.withColumnRenamed("age", "my_new_age")
print(new_df)

+----------+-------+
|my_new_age|   name|
+----------+-------+
|      NULL|Michael|
|        30|   Andy|
|        19| Justin|
+----------+-------+



## Working with Spark SQL queries

In [31]:
#In order to work with SQL queries we need to register spark
#As a temporary SQL view
#We do CreateOrReplace incase we have already existing view it replaces

df.createOrReplaceTempView('people')

In [32]:
#Now you can pass direct SQL queries
results = spark.sql("SELECT * FROM people")

In [33]:
results.show()

+----+-------+
| age|   name|
+----+-------+
|NULL|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [34]:
# Select people with age = 30
results = spark.sql("SELECT * FROM people WHERE age == 30")
results.show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



In [ ]:
# Show


+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



## Working with JSON, CSV and other formats

In [35]:
from pyspark.sql import SparkSession

In [36]:
spark = SparkSession.builder.appName('ops').getOrCreate()

25/04/28 16:19:22 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [ ]:
#infer schema option is available with CSV

In [49]:
appl_stock = spark.read.csv("./data_lab7/appl_stock.csv", header=True, inferSchema=True)
appl_stock.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [ ]:
# Head 3

[Row(Date=datetime.datetime(2010, 1, 4, 0, 0), Open=213.429998, High=214.499996, Low=212.38000099999996, Close=214.009998, Volume=123432400, Adj Close=27.727039),
 Row(Date=datetime.datetime(2010, 1, 5, 0, 0), Open=214.599998, High=215.589994, Low=213.249994, Close=214.379993, Volume=150476200, Adj Close=27.774976000000002),
 Row(Date=datetime.datetime(2010, 1, 6, 0, 0), Open=214.379993, High=215.23, Low=210.750004, Close=210.969995, Volume=138040000, Adj Close=27.333178000000004)]

In [50]:
appl_stock.limit(3).collect()

[Row(Date=datetime.date(2010, 1, 4), Open=213.429998, High=214.499996, Low=212.38000099999996, Close=214.009998, Volume=123432400, Adj Close=27.727039),
 Row(Date=datetime.date(2010, 1, 5), Open=214.599998, High=215.589994, Low=213.249994, Close=214.379993, Volume=150476200, Adj Close=27.774976000000002),
 Row(Date=datetime.date(2010, 1, 6), Open=214.379993, High=215.23, Low=210.750004, Close=210.969995, Volume=138040000, Adj Close=27.333178000000004)]

In [ ]:
%%time
df.head(1)[0]

CPU times: user 0 ns, sys: 16 ms, total: 16 ms
Wall time: 180 ms


Row(Date=datetime.datetime(2010, 1, 4, 0, 0), Open=213.429998, High=214.499996, Low=212.38000099999996, Close=214.009998, Volume=123432400, Adj Close=27.727039)

In [ ]:
df.show()

+--------------------+------------------+------------------+------------------+------------------+---------+------------------+
|                Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+--------------------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04 00:00:...|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05 00:00:...|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06 00:00:...|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07 00:00:...|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08 00:00:...|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902

In [ ]:
#You can pass in conditions 'Close < 500'

+------------------+
|              Open|
+------------------+
|        213.429998|
|        214.599998|
|        214.379993|
|            211.75|
|        210.299994|
|212.79999700000002|
|209.18999499999998|
|        207.870005|
|210.11000299999998|
|210.92999500000002|
|        208.330002|
|        214.910006|
|        212.079994|
|206.78000600000001|
|202.51000200000001|
|205.95000100000001|
|        206.849995|
|        204.930004|
|        201.079996|
|192.36999699999998|
+------------------+
only showing top 20 rows



In [72]:
#Opening price of every stock where Close price is less than 500

In [87]:
appl_stock.select(col("Open")).filter(col("Open") < lit(500)).show()

+------------------+
|              Open|
+------------------+
|        213.429998|
|        214.599998|
|        214.379993|
|            211.75|
|        210.299994|
|212.79999700000002|
|209.18999499999998|
|        207.870005|
|210.11000299999998|
|210.92999500000002|
|        208.330002|
|        214.910006|
|        212.079994|
|206.78000600000001|
|202.51000200000001|
|205.95000100000001|
|        206.849995|
|        204.930004|
|        201.079996|
|192.36999699999998|
+------------------+
only showing top 20 rows



In [ ]:
# Select and show 'Open' and 'Close' with 'Close < 500'

In [86]:
appl_stock.select(col("Open"), col('Close')).filter(col("Close") < lit(500)).show()

+------------------+------------------+
|              Open|             Close|
+------------------+------------------+
|        213.429998|        214.009998|
|        214.599998|        214.379993|
|        214.379993|        210.969995|
|            211.75|            210.58|
|        210.299994|211.98000499999998|
|212.79999700000002|210.11000299999998|
|209.18999499999998|        207.720001|
|        207.870005|        210.650002|
|210.11000299999998|            209.43|
|210.92999500000002|            205.93|
|        208.330002|        215.039995|
|        214.910006|            211.73|
|        212.079994|        208.069996|
|206.78000600000001|            197.75|
|202.51000200000001|        203.070002|
|205.95000100000001|        205.940001|
|        206.849995|        207.880005|
|        204.930004|        199.289995|
|        201.079996|        192.060003|
|192.36999699999998|        194.729998|
+------------------+------------------+
only showing top 20 rows



In [ ]:
# Select and show 'Open' and 'Close' with 'Open < 500'

In [88]:
appl_stock.select(col("Open"), col('Close')).filter(col("Open") < lit(500)).show()

+------------------+------------------+
|              Open|             Close|
+------------------+------------------+
|        213.429998|        214.009998|
|        214.599998|        214.379993|
|        214.379993|        210.969995|
|            211.75|            210.58|
|        210.299994|211.98000499999998|
|212.79999700000002|210.11000299999998|
|209.18999499999998|        207.720001|
|        207.870005|        210.650002|
|210.11000299999998|            209.43|
|210.92999500000002|            205.93|
|        208.330002|        215.039995|
|        214.910006|            211.73|
|        212.079994|        208.069996|
|206.78000600000001|            197.75|
|202.51000200000001|        203.070002|
|205.95000100000001|        205.940001|
|        206.849995|        207.880005|
|        204.930004|        199.289995|
|        201.079996|        192.060003|
|192.36999699999998|        194.729998|
+------------------+------------------+
only showing top 20 rows



In [ ]:
# Select and show 'Volume' with 'Close < 500'

In [89]:
appl_stock.select(col("Volume")).filter(col("Close") < lit(500)).show()

+---------+
|   Volume|
+---------+
|123432400|
|150476200|
|138040000|
|119282800|
|111902700|
|115557400|
|148614900|
|151473000|
|108223500|
|148516900|
|182501900|
|153038200|
|152038600|
|220441900|
|266424900|
|466777500|
|430642100|
|293375600|
|311488100|
|187469100|
+---------+
only showing top 20 rows



In [91]:
# Multiple conditions 'Close' < 200  and 'Open' > 200
appl_stock.filter((col("Close") < lit(200)) & (col("Open") > lit(200))).show()

+----------+------------------+----------+----------+----------+---------+------------------+
|      Date|              Open|      High|       Low|     Close|   Volume|         Adj Close|
+----------+------------------+----------+----------+----------+---------+------------------+
|2010-01-22|206.78000600000001|207.499996|    197.16|    197.75|220441900|         25.620401|
|2010-01-28|        204.930004|205.500004|198.699995|199.289995|293375600|25.819922000000002|
|2010-01-29|        201.079996|202.199995|190.250002|192.060003|311488100|         24.883208|
+----------+------------------+----------+----------+----------+---------+------------------+



In [ ]:
# Multiple conditions 'Close' < 200  and 'Open' <= 200

In [92]:
appl_stock.filter((col("Close") < lit(200)) & (col("Open") <= lit(200))).show()

+----------+------------------+------------------+------------------+------------------+---------+------------------+
|      Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+----------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-02-01|192.36999699999998|             196.0|191.29999899999999|        194.729998|187469100|         25.229131|
|2010-02-02|        195.909998|        196.319994|193.37999299999998|        195.859997|174585600|25.375532999999997|
|2010-02-03|        195.169994|        200.200003|        194.420004|        199.229994|153832000|25.812148999999998|
|2010-02-04|        196.730003|        198.370001|        191.570005|        192.050003|189413000|         24.881912|
|2010-02-05|192.63000300000002|             196.0|        190.850002|        195.460001|212576700|25.323710000000002|
|2010-02-08|        195.690006|197.88000300000002|      

In [93]:
appl_stock.filter(appl_stock["Low"] == 197.16).show()

+----------+------------------+----------+------+------+---------+---------+
|      Date|              Open|      High|   Low| Close|   Volume|Adj Close|
+----------+------------------+----------+------+------+---------+---------+
|2010-01-22|206.78000600000001|207.499996|197.16|197.75|220441900|25.620401|
+----------+------------------+----------+------+------+---------+---------+



## GroupBy, Aggregate and Sorting Functions

In [94]:
spark = SparkSession.builder.appName('aggs').getOrCreate()

25/04/28 17:01:33 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [95]:
df = spark.read.csv("./data_lab7/sales_info.csv", inferSchema=True, header=True)

In [96]:
df.count()

12

In [97]:
df.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: double (nullable = true)



In [98]:
df.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



In [ ]:
# groupBy 'Company'and show the counting

+-------+-----+
|Company|count|
+-------+-----+
|   APPL|    4|
|   GOOG|    3|
|     FB|    2|
|   MSFT|    3|
+-------+-----+



In [101]:
df.groupBy('Company').count().show()

+-------+-----+
|Company|count|
+-------+-----+
|   APPL|    4|
|   GOOG|    3|
|     FB|    2|
|   MSFT|    3|
+-------+-----+



In [ ]:
# groupBy 'Company'and show the mea

In [113]:
df.groupBy("Company").mean("Sales").show()

+-------+-----------------+
|Company|       avg(Sales)|
+-------+-----------------+
|   APPL|            370.0|
|   GOOG|            220.0|
|     FB|            610.0|
|   MSFT|322.3333333333333|
+-------+-----------------+



In [ ]:
# groupBy 'Company'and show the max

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+



In [114]:
df.groupBy("Company").max("Sales").show()

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+



In [ ]:
# groupBy 'Company'and show the min

+-------+----------+
|Company|min(Sales)|
+-------+----------+
|   APPL|     130.0|
|   GOOG|     120.0|
|     FB|     350.0|
|   MSFT|     124.0|
+-------+----------+



In [115]:
df.groupBy("Company").min("Sales").show()

+-------+----------+
|Company|min(Sales)|
+-------+----------+
|   APPL|     130.0|
|   GOOG|     120.0|
|     FB|     350.0|
|   MSFT|     124.0|
+-------+----------+



In [ ]:
## May be you dont need group by, you need something
#like average sales per company

+----------+
|sum(Sales)|
+----------+
|    4327.0|
+----------+



In [119]:
from pyspark.sql.functions import sum

df.select(sum(col("Sales"))).show()

+----------+
|sum(Sales)|
+----------+
|    4327.0|
+----------+



In [127]:
from pyspark.sql.functions import countDistinct, avg, stddev, stddev_samp

In [ ]:
#Counts distinct sales value

+---------------------+
|count(DISTINCT Sales)|
+---------------------+
|                   11|
+---------------------+



In [121]:
df.select(countDistinct(col("Sales"))).show()

+---------------------+
|count(DISTINCT Sales)|
+---------------------+
|                   11|
+---------------------+



+-----------------+
|       avg(Sales)|
+-----------------+
|360.5833333333333|
+-----------------+



In [122]:
df.select(avg(col("Sales"))).show()

+-----------------+
|       avg(Sales)|
+-----------------+
|360.5833333333333|
+-----------------+



In [ ]:
#Alias - Give the column name appropriate name

+-----------------+
|    Average Sales|
+-----------------+
|360.5833333333333|
+-----------------+



In [125]:
df.select(avg(col("Sales")).alias("Average Sales")).show()

+-----------------+
|    Average Sales|
+-----------------+
|360.5833333333333|
+-----------------+



+------------------+
|stddev_samp(Sales)|
+------------------+
|250.08742410799007|
+------------------+



In [128]:
df.select(stddev_samp(col("Sales"))).show()

+------------------+
|stddev_samp(Sales)|
+------------------+
|250.08742410799007|
+------------------+



In [129]:
#Formatting the number
from pyspark.sql.functions import format_number

In [130]:
sales_std = df.select(stddev('Sales').alias('std'))

In [132]:
sales_std.show()

+------------------+
|               std|
+------------------+
|250.08742410799007|
+------------------+



In [ ]:
#number of decimal places I want to show

+---------------------+
|format_number(std, 2)|
+---------------------+
|               250.09|
+---------------------+



In [138]:
df.select(format_number(stddev(col("Sales")), 2)).show()

+-------------------------------+
|format_number(stddev(Sales), 2)|
+-------------------------------+
|                         250.09|
+-------------------------------+



In [ ]:
#Ordering the columns, default is ascending order

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|Charlie|120.0|
|   MSFT|    Amy|124.0|
|   APPL|  Linda|130.0|
|   GOOG|    Sam|200.0|
|   MSFT|Vanessa|243.0|
|   APPL|   John|250.0|
|   GOOG|  Frank|340.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   MSFT|   Tina|600.0|
|   APPL|   Mike|750.0|
|     FB|   Carl|870.0|
+-------+-------+-----+



In [140]:
df.orderBy(col("Sales")).show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|Charlie|120.0|
|   MSFT|    Amy|124.0|
|   APPL|  Linda|130.0|
|   GOOG|    Sam|200.0|
|   MSFT|Vanessa|243.0|
|   APPL|   John|250.0|
|   GOOG|  Frank|340.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   MSFT|   Tina|600.0|
|   APPL|   Mike|750.0|
|     FB|   Carl|870.0|
+-------+-------+-----+



In [ ]:
#Ordering in descending

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|870.0|
|   APPL|   Mike|750.0|
|   MSFT|   Tina|600.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   GOOG|  Frank|340.0|
|   APPL|   John|250.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   APPL|  Linda|130.0|
|   MSFT|    Amy|124.0|
|   GOOG|Charlie|120.0|
+-------+-------+-----+



In [145]:
df.orderBy(col("Sales"), ascending=False).show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|870.0|
|   APPL|   Mike|750.0|
|   MSFT|   Tina|600.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   GOOG|  Frank|340.0|
|   APPL|   John|250.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   APPL|  Linda|130.0|
|   MSFT|    Amy|124.0|
|   GOOG|Charlie|120.0|
+-------+-------+-----+



## Dealing with Missing Data in Spark

In [146]:
spark = SparkSession.builder.appName('miss').getOrCreate()

25/04/28 17:37:43 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [148]:
df = spark.read.csv("./data_lab7/ContainsNull.csv", header=True, inferSchema=True)

In [149]:
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| NULL|
|emp2| NULL| NULL|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [ ]:
#You can drop all the missing data

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [157]:
df.dropna().show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [ ]:
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [ ]:
#When we specify thresh=2, it looks at the row if it contains more than 2 non-null values, it drops

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [155]:
df.dropna(thresh=2).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| NULL|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [ ]:
#By default how is set to 'any', meaning drop if there are any null values

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [164]:
df.dropna(how="any").show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [ ]:
#We can change the how to 'all', if row has all null values only then drop

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [163]:
df.dropna(how="all").show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| NULL|
|emp2| NULL| NULL|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [ ]:
#The subset method will only drop if Sales colum has null values

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [162]:
df.dropna(subset="Sales").show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [ ]:
df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sales: double (nullable = true)



In [ ]:
#We can also fill NULL values

+----+----------+-----+
|  Id|      Name|Sales|
+----+----------+-----+
|emp1|      John| null|
|emp2|FILL VALUE| null|
|emp3|FILL VALUE|345.0|
|emp4|     Cindy|456.0|
+----+----------+-----+



In [165]:
df.fillna(value="FILL VALUE").show()

+----+----------+-----+
|  Id|      Name|Sales|
+----+----------+-----+
|emp1|      John| NULL|
|emp2|FILL VALUE| NULL|
|emp3|FILL VALUE|345.0|
|emp4|     Cindy|456.0|
+----+----------+-----+



In [ ]:
#Spark is smart enough to recognize string datatype and only fill
#in string datatype column, similar thing it does for numeric values

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|  0.0|
|emp2| null|  0.0|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [166]:
df.fillna(value=0.0).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|  0.0|
|emp2| NULL|  0.0|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [167]:
from pyspark.sql.functions import mean

In [168]:
mean_val = df.select(mean(df['Sales'])).collect()

In [169]:
mean_val[0]

Row(avg(Sales)=400.5)

In [170]:
mean_val[0][0]

400.5

In [171]:
mean_sales = mean_val[0][0]

In [172]:
df.na.fill(mean_sales, ['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| NULL|400.5|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [ ]:
#Doing all this in one line

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [180]:
df.fillna(subset="Sales", value=df.select(mean(col("Sales"))).collect()[0][0]).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| NULL|400.5|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



## Dates and Timestamps

In [181]:
spark = SparkSession.builder.appName('dates').getOrCreate()

25/04/28 18:06:33 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [183]:
df = spark.read.csv("./data_lab7/appl_stock.csv", header=True, inferSchema=True)

In [184]:
df.count()

1762

In [185]:
df.head(1)
#datetime object in Date column, this looks messy here

[Row(Date=datetime.date(2010, 1, 4), Open=213.429998, High=214.499996, Low=212.38000099999996, Close=214.009998, Volume=123432400, Adj Close=27.727039)]

In [186]:
df.select(['Date', 'Open']).show()

+----------+------------------+
|      Date|              Open|
+----------+------------------+
|2010-01-04|        213.429998|
|2010-01-05|        214.599998|
|2010-01-06|        214.379993|
|2010-01-07|            211.75|
|2010-01-08|        210.299994|
|2010-01-11|212.79999700000002|
|2010-01-12|209.18999499999998|
|2010-01-13|        207.870005|
|2010-01-14|210.11000299999998|
|2010-01-15|210.92999500000002|
|2010-01-19|        208.330002|
|2010-01-20|        214.910006|
|2010-01-21|        212.079994|
|2010-01-22|206.78000600000001|
|2010-01-25|202.51000200000001|
|2010-01-26|205.95000100000001|
|2010-01-27|        206.849995|
|2010-01-28|        204.930004|
|2010-01-29|        201.079996|
|2010-02-01|192.36999699999998|
+----------+------------------+
only showing top 20 rows



In [187]:
from pyspark.sql.functions import (
    dayofmonth,
    hour,
    dayofyear,
    month,
    year,
    weekofyear,
    format_number,
    date_format,
)

In [188]:
# Whenever you are using any pyspark imported function,
# you can directly use them in select on your column
df.select(dayofmonth(df["Date"])).show()

+----------------+
|dayofmonth(Date)|
+----------------+
|               4|
|               5|
|               6|
|               7|
|               8|
|              11|
|              12|
|              13|
|              14|
|              15|
|              19|
|              20|
|              21|
|              22|
|              25|
|              26|
|              27|
|              28|
|              29|
|               1|
+----------------+
only showing top 20 rows



In [189]:
df.select(hour(df["Date"])).show()

+----------+
|hour(Date)|
+----------+
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
+----------+
only showing top 20 rows



In [ ]:
df.select(month(df['Date'])).show()

+-----------+
|month(Date)|
+-----------+
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          2|
+-----------+
only showing top 20 rows



In [ ]:
## May be you want to know average closing price per year

+----------+
|year(Date)|
+----------+
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
+----------+
only showing top 20 rows



In [190]:
df.select(year(col("Date"))).show()

+----------+
|year(Date)|
+----------+
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
+----------+
only showing top 20 rows



In [ ]:
# Take the year

+--------------------+------------------+------------------+------------------+------------------+---------+------------------+----+
|                Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|Year|
+--------------------+------------------+------------------+------------------+------------------+---------+------------------+----+
|2010-01-04 00:00:...|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|2010|
|2010-01-05 00:00:...|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|2010|
|2010-01-06 00:00:...|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|2010|
|2010-01-07 00:00:...|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|2010|
|2010-01-08 00:00:...|        210.299994|        212.000006|209.06000

In [193]:
df.withColumn("Year", year(col("Date"))).show()

+----------+------------------+------------------+------------------+------------------+---------+------------------+----+
|      Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|Year|
+----------+------------------+------------------+------------------+------------------+---------+------------------+----+
|2010-01-04|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|2010|
|2010-01-05|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|2010|
|2010-01-06|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|2010|
|2010-01-07|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|2010|
|2010-01-08|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|2010|
|2010-01-11|212.

In [ ]:
# Create new column `Year`

+----+------------------+------------------+------------------+------------------+--------------------+------------------+---------+
|Year|         avg(Open)|         avg(High)|          avg(Low)|        avg(Close)|         avg(Volume)|    avg(Adj Close)|avg(Year)|
+----+------------------+------------------+------------------+------------------+--------------------+------------------+---------+
|2015|120.17575393253965|121.24452385714291| 118.8630954325397|120.03999980555547|  5.18378869047619E7|115.96740080555561|   2015.0|
|2013| 473.1281355634922| 477.6389272301587|468.24710264682557| 472.6348802857143|          1.016087E8| 62.61798788492063|   2013.0|
|2014| 295.1426195357143|297.56103184523823| 292.9949599801587| 295.4023416507935| 6.315273055555555E7| 87.63583323809523|   2014.0|
|2012|     576.652720788| 581.8254008040001| 569.9211606079999| 576.0497195640002|       1.319642044E8| 74.81383696800002|   2012.0|
|2016|104.50777772619044| 105.4271825436508|103.69027771825397|104.60

In [201]:
df = df.withColumn("Year", year(col("Date")))
df.groupBy("Year").agg(
    avg("Open"),
    avg("High"),
    avg("Low"),
    avg("Close"),
    avg("Volume"),
    avg("Adj Close"),
    avg("Year"),
).show()

+----+------------------+------------------+------------------+------------------+--------------------+------------------+---------+
|Year|         avg(Open)|         avg(High)|          avg(Low)|        avg(Close)|         avg(Volume)|    avg(Adj Close)|avg(Year)|
+----+------------------+------------------+------------------+------------------+--------------------+------------------+---------+
|2015|120.17575393253965|121.24452385714291| 118.8630954325397|120.03999980555547|  5.18378869047619E7|115.96740080555561|   2015.0|
|2013| 473.1281355634922| 477.6389272301587|468.24710264682557| 472.6348802857143|          1.016087E8| 62.61798788492063|   2013.0|
|2014| 295.1426195357143|297.56103184523823| 292.9949599801587| 295.4023416507935| 6.315273055555555E7| 87.63583323809523|   2014.0|
|2012|     576.652720788| 581.8254008040001| 569.9211606079999| 576.0497195640002|       1.319642044E8| 74.81383696800002|   2012.0|
|2016|104.50777772619044| 105.4271825436508|103.69027771825397|104.60

In [ ]:
# Groupby `Year` and average the Close price

+------------------+----+
|        avg(Close)|Year|
+------------------+----+
|120.03999980555547|2015|
| 472.6348802857143|2013|
| 295.4023416507935|2014|
| 576.0497195640002|2012|
|104.60400786904763|2016|
| 259.8424600000002|2010|
|364.00432532142867|2011|
+------------------+----+



In [204]:
rs = df.groupBy("Year").agg(avg("Close"))
rs.select("avg(Close)", "Year").show()

+------------------+----+
|        avg(Close)|Year|
+------------------+----+
|120.03999980555547|2015|
| 472.6348802857143|2013|
| 295.4023416507935|2014|
| 576.0497195640002|2012|
|104.60400786904763|2016|
| 259.8424600000002|2010|
|364.00432532142867|2011|
+------------------+----+



## Thank you!